In [5]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import sys
import os
import torch as pt
from tqdm import tqdm

# Add the project root
sys.path.append(os.path.abspath('../../'))

from src.useful_functions import read_dirs_paths
from src.isokann.modules3 import *

# For matplotlib
font = {'size'   : 10}
plt.rc('font', **font)
in2cm = 1/2.54  # centimeters in inches

# For reproducibility
np.random.seed(0)
pt.manual_seed(0)

# Read directory paths
read_dirs_paths('dir_paths.txt', globals())

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")
print("")
print(device)


Created variables:
out_trajectories = output/trajectories/
out_isokann = output/isokann/
out_mokito = output/mokito/

cuda


In [6]:
# Load initial and final states and convert to torch
X0       = pt.tensor(np.load(out_trajectories + 'X0.npy'), dtype=pt.float32).to(device)
Xt       = pt.tensor(np.load(out_trajectories + 'Xt.npy'), dtype=pt.float32).to(device)

Npoints = X0.shape[0]
Ndims   = X0.shape[1]


print(Xt.shape)

torch.Size([4000, 10, 2])


In [7]:
NN_epochs = [5, 10 ,15]


NN_nodes =          [ (Ndims, 64, 1),
                      (Ndims, 128, 1),
                      (Ndims, 64, 32, 1)]

NN_lr = [ 0.01,
          0.001,
          0.0001]

NN_wd  = [ 0.001,
           0.0001,
           0.00001]

NN_bs  = [50, 100, 200]

NN_patience = [2, 3, 5]

NN_act_fun = ['sigmoid']

best_hyperparams, best_val_loss = random_search(X0,
                                                Xt,
                                                NN_epochs,
                                                NN_nodes,
                                                NN_lr,
                                                NN_wd,
                                                NN_bs,
                                                NN_patience,
                                                NN_act_fun,
                                                search_iterations=5,
                                                test_size = 0.2)

print("The best hyperparameters are:", best_hyperparams)
print("The best validation loss is:",  best_val_loss)


  0%|          | 0/5 [00:00<?, ?it/s]

 
Nepochs = 15
Nodes = [  2 128   1]
Learning rate = 0.001
Weight decay = 1e-05
Batch size = 100
Patience = 2
Activation function = sigmoid


 20%|██        | 1/5 [00:06<00:24,  6.04s/it]

Validation loss: tensor(0.0007, device='cuda:0')
Convergence: 1.0005796241869787
 
Nepochs = 5
Nodes = [ 2 64 32  1]
Learning rate = 0.001
Weight decay = 1e-05
Batch size = 200
Patience = 5
Activation function = sigmoid


 40%|████      | 2/5 [00:19<00:31, 10.64s/it]

...

Validation loss: tensor(0.0008, device='cuda:0')
Convergence: 1.0005326012729867
 
Nepochs = 5
Nodes = [ 2 64 32  1]
Learning rate = 0.01
Weight decay = 0.001
Batch size = 50
Patience = 5
Activation function = sigmoid


 80%|████████  | 4/5 [00:29<00:06,  6.81s/it]

Validation loss: tensor(0.0009, device='cuda:0')
Convergence: 1.0005583918255858
 
Nepochs = 5
Nodes = [ 2 64  1]
Learning rate = 0.001
Weight decay = 1e-05
Batch size = 100
Patience = 2
Activation function = sigmoid


100%|██████████| 5/5 [00:34<00:00,  6.89s/it]

Validation loss: tensor(0.0007, device='cuda:0')
Convergence: 1.0008898121982133
The best hyperparameters are: {'Nepochs': 15, 'nodes': array([  2, 128,   1]), 'learning_rate': 0.001, 'weight_decay': 1e-05, 'batch_size': 100, 'patience': 2, 'act_fun': 'sigmoid'}
The best validation loss is: tensor(0.0007, device='cuda:0')


In [8]:
import pickle
# Save to a file
with open(out_isokann + 'hyperparameters.pkl', 'wb') as file:
    pickle.dump(best_hyperparams, file)